# Analysis of the Evian water

The tutorial considers the problem dedicated to checking the quality of the Evian water using Reaktoro with custom-defined chemical composition.

![title](../../images/evian-chemical-water-composition.png)

We start from setting a chemical system, constraints of temperature, pressure, and pH, and their values:

In [17]:
from reaktoro import *
from math import *

db = SupcrtDatabase("supcrtbl")

# Create an aqueous phase automatically selecting all species with provided elements
aqueousphase = AqueousPhase(speciate("C Ca Cl H K Mg N Na O S Si"))
aqueousphase.setActivityModel(chain(
    ActivityModelHKF(),
    ActivityModelDrummond("CO2"),
))

minerals = MineralPhases("Calcite Dolomite Quartz")

# Create the chemical system
system = ChemicalSystem(db, aqueousphase, minerals)

specs = EquilibriumSpecs(system)
specs.temperature()
specs.pressure()
specs.pH()

solver = EquilibriumSolver(specs)

conditions = EquilibriumConditions(specs)
conditions.temperature(25.0, "celsius")
conditions.pressure(1.0, "atm")
conditions.pH(7.2)

The composition of initial chemical state is taken from the plot with a label of Evian water. However, since the concentration of the species in Evian water are defined in milligrams per liter (mg/L), we need to convert these values to moles using species molar masses. The latter can be obtained using function `db.species("H+").molarMass()`. Also, to convert `mg` into `kg`, we multiply the values by 1e-6. We also equilibrate the given chemical state.

In [18]:
state = ChemicalState(system)

state.set("H2O(aq)", 1.0, "kg")
state.set("Ca+2" , 80 * 1e-6 / db.species("Ca+2").molarMass(), "mol")  # 80 * 1e-6 kg / (MW(Ca++) kg / mol)
state.set("Cl-"  , 6.8 * 1e-6 / db.species("Cl-").molarMass(), "mol")  # 6.8 * 1e-6 kg / (MW(Cl-) kg / mol)
state.set("HCO3-", 350 * 1e-6 / db.species("HCO3-").molarMass(), "mol")  # 350 * 1e-6 kg / (MW(HCO3-) kg / mol)
state.set("Mg+2" , 26 * 1e-6 / db.species("Mg+2").molarMass(), "mol")   # 26 * 1e-6 kg / (MW(Mg++) kg / mol)
state.set("NO3-" , 3.7 * 1e-6 / db.species("NO3-").molarMass(), "mol")  # 3.7 * 1e-6 kg / (MW(NO3-) kg / mol)
state.set("K+"   , 1 * 1e-6 / db.species("K+").molarMass(), "mol")  # 1 * 1e-6 kg / (MW(K+) kg / mol)
state.set("Na+"  , 6.5 * 1e-6 / db.species("Na+").molarMass(), "mol")  # 6.5 * 1e-6 kg / (MW(Na+) kg / mol)
state.set("SO4-2", 12.6 * 1e-6 / db.species("S2O4-2").molarMass(), "mol")  # 12.6 * 1e-6 kg / (MW(SO4--) kg / mol)
state.set("SiO2(aq)" , 15 * 1e-6 / db.species("SiO2(aq)").molarMass(), "mol")  # 15 * 1e-6 kg / (MW(SiO2) kg/ mol

solver.solve(state, conditions)

In order to obtain saturation indices of the carbonates and quartz, we need to access aqueous properties of the
calculated chemical state. The saturation index is defined as a log10 of the ratio of equilibrium constant and
reaction quotient. It is 0 for minerals that are *precipitated* (i.e., in equilibrium with the solution), SI > 0 for
*supersaturated minerals*, and SI < 0 for *undersaturated minerals*.

In [19]:
# Calculate saturation indices
props = AqueousProps(state)
print("SI (Calcite) = ", props.saturationIndexLg('Calcite'))
print("SI (Dolomite) = ", props.saturationIndexLg('Dolomite'))
print("SI (Quartz) = ", props.saturationIndexLg('Quartz'))

SI (Calcite) =  -0.0197293
SI (Dolomite) =  -5.71062e-13
SI (Quartz) =  -0.155618


Based on the results above, the water is *saturated with dolomite*, whereas calcite and quartz are *undersaturated*. This agrees with the amount of these minerals after in the resulting chemical state:

In [20]:
# Calculate minerals amounts
print(f"Calcite  amount: {state.speciesAmount('Calcite') } mol")
print(f"Dolomite amount: {state.speciesAmount('Dolomite')} mol")
print(f"Quartz   amount: {state.speciesAmount('Quartz')  } mol")

Calcite  amount: 2.20126e-15 mol
Dolomite amount: 0.000576783 mol
Quartz   amount: 2.79077e-16 mol
